In [100]:
#import libraries
#The "TARGET" column is the variable to predict. It equals 1 for unsatisfied customers and 0 for satisfied customers.

#Models
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn import svm
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler


#Transformation
import numpy as np
import pandas as pd
import random

#Graphics
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'xgboost'

In [2]:
#Loading the train Data set
df_train = pd.read_csv("DataSet/train.csv")
df_test = pd.read_csv("DataSet/Test.csv")
Y_test = pd.read_csv("DataSet/sample_submission.csv")
df_train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [3]:
#Basic statistics
df_train.describe()


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [4]:
#76020 variables and 371 observations. 
df_train.shape

(76020, 371)

In [5]:
#Check types for all collumns - Only Int and Float
dataTypeSeries = set(df_train.dtypes)
dataTypeSeries

{dtype('int64'), dtype('float64')}

In [6]:
#Checking the balance of the dataSET, as indicated, only 3% of my Dataset has the classification of satisfied client.

df=pd.DataFrame(df_train["ID"].groupby(df_train["TARGET"]).count())
Num_sat=int(df.iloc[1])

df_train["TARGET"].mean()


0.0395685345961589

In [7]:
#Moving ID to index, this way the column ID will be "removed" from the DataFrame
df_train.index=df_train["ID"]
df_train.drop("ID", axis=1, inplace=True)

In [8]:
df_train.shape

(76020, 370)

In [9]:
#Lista dos index com clientes satisfeitos e insatisfeitos
Index_Sat=np.array(df_train.loc[df_train.TARGET>0].index)
Index_InSat=np.array(df_train.loc[df_train.TARGET==0].index)
df_train.loc[Index_InSat].head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
ID,,,,,,,,,,,,,,,,,,,,,
1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [10]:
#Gerando uma lista de itens randomicos com index das pessoas insatisfeitas para que temos as mesmas observações de 
#pessoas satisfeitas e insatisfeitas
random_Index_Inst=random.sample(list(Index_InSat),Num_sat)
df_train.loc[random_Index_Inst].head(5)

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
ID,,,,,,,,,,,,,,,,,,,,,
145303,2,31,0.0,3.0,75.15,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,112962.72,0
27097,2,36,0.0,387.3,387.30,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31720.83,0
7005,2,33,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,173771.40,0
12407,2,24,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,324032.76,0
84526,2,26,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,528859.59,0


In [11]:
#Now the Dataset has the same number of observations for 1 and 0.
List_dataset = [df_train.loc[Index_Sat],df_train.loc[random_Index_Inst]]
New_data_train = pd.concat(List_dataset)
New_data_train[["var3", "TARGET"]].groupby("TARGET").count()

,var3
TARGET,
0,3008
1,3008


In [12]:
# Now the median of observation for the classification is 50%
New_data_train["TARGET"].mean()

0.5

In [13]:
#Separado em dados em X, Y de treino e de dados de test
#Dataset Treino
Y_treino = New_data_train["TARGET"].values
New_data_train.drop("TARGET", axis=1, inplace=True)
X_treino = New_data_train
print(X_treino.shape, Y_treino.shape)


(6016, 369) (6016,)


In [14]:
#The test Dataset in this project was given with already splited in X and Y.
#Here we will drop the column ID

#Mudanças para X_Test
df_test.index = df_test["ID"]
df_test.drop("ID", axis=1, inplace=True)
X_test = df_test
#Mudanças para Y_test
Y_test.drop("ID", axis=1, inplace=True)
Y_test=Y_test["TARGET"].values


In [15]:
print(X_treino.shape, Y_treino.shape,X_test.shape, Y_test.shape)

(6016, 369) (6016,) (75818, 369) (75818,)


# Normalização dos dados

In [30]:
#Normalização com o normalizer

#Normalização de treino
x_treino_array=X_treino.values
scaler = Normalizer().fit(x_treino_array)
normalizedX = scaler.transform(x_treino_array)
X_treino_norm=pd.DataFrame(normalizedX,columns=X_treino.columns, index=X_treino.index)

#Normalização de test
x_test_array=X_test.values
scaler = Normalizer().fit(x_test_array)
normalizedX = scaler.transform(x_test_array)
X_test_norm=pd.DataFrame(normalizedX,columns=X_test.columns, index=X_test.index)


In [33]:
print(X_treino_norm.shape,X_test_norm.shape)

(6016, 369) (75818, 369)


# Redução da dimensionalidade

In [34]:
#Utilizando o PCA para reduzir a dimensionalidade
pca = decomposition.PCA(n_components = 100, 
                        whiten = True, 
                        svd_solver = 'randomized')

In [35]:
# O PCA é um algoritmo de aprendizagem não supervisionada
pca.fit(X_treino_norm)

PCA(n_components=100, svd_solver='randomized', whiten=True)

In [36]:
# E então aplicamos o modelo PCA nos dados de treino e de teste
X_treino_pca = pca.transform(X_treino_norm)
X_test_pca = pca.transform(X_test_norm)

In [37]:
# Shape
print(X_treino_pca.shape)
print(X_test_pca.shape)

(6016, 100)
(75818, 100)


# Criação do Modelos 

SVM - Suport vetor Machine

In [95]:
#ML com SVM
# Cria o modelo
modelo_svm = svm.SVC(C = 5., gamma = 0.001)
# Treinamento do modelo
modelo_svm.fit(X_treino_pca, Y_treino)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(C=5.0, gamma=0.001)

In [96]:
#Previsões
previsoes = modelo_svm.predict(X_test_pca)

In [97]:
print(modelo_svm.score(X_test_pca, Y_test))

0.7445329605106966


In [99]:
data_dmatrix = xgb.DMatrix(data=X_treino_pca,label=Y_treino)

NameError: name 'xgb' is not defined